# Механізм уваги для обробки тестів

Механізми уваги зробили революцію в обробці природної мови (NLP), дозволивши моделям зосередитися на найбільш релевантних частинах вхідної послідовності. У цьому посібнику розглядаються поняття уваги, багатоголової уваги та кодерів-трансформерів у PyTorch для задач NLP.

Розуміння уваги:

Уявіть, що ви читаєте речення. Ви не приділяєте однакову увагу кожному слову, а зосереджуєтесь на найбільш важливих, щоб зрозуміти зміст. Аналогічно, увага в моделях НЛП дозволяє моделі концентруватися на певних частинах вхідної послідовності (наприклад, реченні), які є найбільш важливими для конкретного завдання, наприклад, аналізу настрою або машинного перекладу.


<div>
<img src="https://preview.redd.it/nrd3yld06rr91.png?width=761&format=png&auto=webp&s=76b11148418849a21304943898526dbdfb60052c" width="500"/>
</div>

## Ключові поняття:

* **Вектори запиту, ключа та значення:** Модель отримує на вхід 3 тензора: запит, ключ і значення. Вони відображають різні аспекти елементів послідовності.
* **Ймовірність уваги:** Модель обчислює оцінку для кожної пари елементів у послідовності. Ця оцінка й відображає, наскільки важливим є один елемент (на основі його вектора ключа) для поточного елемента (на основі його вектора запиту).
* **Зважена сума:** Використовуючи оцінки уваги, модель створює зважену суму векторів значень, ефективно фокусуючись на найбільш релевантних частинах послідовності.

**Переваги уваги:** 

* **Довгострокові залежності:** Увага допомагає вловлювати довгострокові залежності в тексті, де віддалені слова можуть бути семантично пов'язані між собою. Це має вирішальне значення для таких завдань, як відповіді на запитання або аналіз настроїв.
* **Розпаралелювання:** Обчислення уваги можна ефективно розпаралелити, що робить їх придатними для навчання на великих наборах даних за допомогою графічних процесорів.

![](https://www.researchgate.net/publication/356271104/figure/fig5/AS:1090999354961922@1637125924695/The-general-process-of-attention-mechanism.png)

# Багатоголова увага

Стандартний механізм уваги фокусується на одному аспекті взаємозв'язків між елементами. Багатоголова увага вирішує цю проблему, виконуючи увагу з декількох «голів», кожна з яких вивчає різні представлення взаємозв'язків. Це дозволяє моделі відображати більш глибоке розуміння вхідної послідовності.

![](https://miro.medium.com/v2/resize:fit:1400/1*PiZyU-_J_nWixsTjXOUP7Q.png)

# Модель Трансформер

Трансформаторна архітектура, популярний вибір для завдань НЛП, значною мірою покладається на механізми уваги. Кодер Transformer використовує шари багатоголової уваги з наступними мережами прямого поширення. Ці шари дозволяють моделі вивчати складні взаємозв'язки між елементами вхідної послідовності.

![](https://quantdare.com/wp-content/uploads/2021/11/transformer_arch.png)

# Читання даних

In [1]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
import pandas as pd

df = pd.read_csv("/kaggle/input/bbc-full-text-document-classification/bbc_data.csv")
df.head()

,data,labels
0,Musicians to tackle US red tape Musicians gro...,entertainment
1,"U2s desire to be number one U2, who have won ...",entertainment
2,Rocker Doherty in on-stage fight Rock singer ...,entertainment
3,Snicket tops US box office chart The film ada...,entertainment
4,"Oceans Twelve raids box office Oceans Twelve,...",entertainment


In [16]:
from torch.utils.data import Dataset
from sklearn.preprocessing import LabelEncoder
import torchtext


class MyDataset(Dataset):
    def __init__(self, X, y, encoding_dim, max_len=100):
        self.X = X
        self.y = y
        self.max_len = max_len
        
        self.label_encoder = LabelEncoder().fit(y)
        self.vocab = torchtext.vocab.GloVe(name='6B', dim=encoding_dim)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        label = self.label_encoder.transform([self.y.iloc[idx]])
        label = torch.tensor(label)
        
        text = self.X.iloc[idx]
        tokens = text.split()
        
        if len(tokens) > self.max_len:
            tokens = tokens[:self.max_len]
        else:
            diff = self.max_len - len(tokens)
            
            tokens += ['<pad>'] * diff
        
        X = self.vocab.get_vecs_by_tokens(tokens, lower_case_backup=True)
        
        return X, label[0]
    
    
dataset = MyDataset(df['data'], df['labels'], 50)

In [4]:
dataset[0][0].shape

torch.Size([100, 50])

In [17]:
from torch.utils.data import DataLoader

batch_size = 16
train_dl = DataLoader(dataset,  # датасет з даними
                        batch_size=batch_size,  # кількість даних в одному пакеті
                        shuffle=True)

NameError: name 'model' is not defined

In [18]:
from torch import nn
import torch.nn.functional as F
import numpy as np


class TextClassifier(nn.Module):
    def __init__(self, encoding_dim, num_classes):
        super().__init__()

        self.encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=encoding_dim, nhead=2, batch_first=True, dim_feedforward=64),
            num_layers=1
        )

        self.flatten = nn.Flatten()

        self.linear1 = nn.Linear(100*50, num_classes)

    def forward(self, x):
        out = self.encoder(x)
        out = self.flatten(out)
        out = self.linear1(out)
        return out


    def predict(self, X, device='cpu'):
        X = torch.FloatTensor(np.array(X)).to(device)

        with torch.no_grad():
            y_pred = F.softmax(self.forward(X), dim=-1)

        return y_pred.cpu().numpy()


model = TextClassifier(50, 5).to(device)
model

TextClassifier(
  (encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=50, out_features=50, bias=True)
        )
        (linear1): Linear(in_features=50, out_features=64, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=64, out_features=50, bias=True)
        (norm1): LayerNorm((50,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((50,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear1): Linear(in_features=5000, out_features=5, bias=True)
)

NameError: name 'X' is not defined

In [19]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [21]:
X, y = next(iter(train_dl))

model(X.to(device))

tensor([[ 0.5110, -0.5174, -0.5850,  0.3872, -0.7153],
        [-0.4042, -0.6381,  1.3053,  0.1748, -0.1258],
        [-0.1710, -0.7105,  1.2146,  0.1666, -0.4207],
        [ 0.0259, -0.9879,  0.2146,  0.1476, -0.6036],
        [-0.9961, -0.7443,  0.7241, -0.2904, -1.0233],
        [ 0.7888, -0.4488, -0.1858, -0.4195, -0.5415],
        [-0.1221, -1.2209,  0.8606, -0.2441, -0.8110],
        [-0.4983, -1.0048, -0.0506,  0.5037, -0.8385],
        [ 0.8026, -0.5030, -0.0791, -0.0119, -0.4100],
        [-0.9414, -0.5763,  0.8781,  0.2530, -1.0987],
        [-0.0884, -0.3518,  0.4113, -0.1300, -0.3059],
        [-0.1724, -0.6278,  0.8081,  0.1573, -0.1922],
        [-0.9416, -0.4749,  1.8490,  0.4080, -0.4054],
        [-0.1918, -0.6672,  1.0119, -0.1743, -0.6668],
        [ 0.2240, -0.4512,  1.1516,  0.5543, -0.2016],
        [ 0.0121, -0.3552,  0.5433, -0.3864, -0.2506]], device='cuda:0',
       grad_fn=<AddmmBackward0>)

In [22]:
import time

def train(model, optimizer, loss_fn, train_dl, val_dl,
          metrics=None, metrics_name=None, epochs=20, device='cpu', task='regression'):
    '''
    Runs training loop for classification problems. Returns Keras-style
    per-epoch history of loss and accuracy over training and validation data.

    Parameters
    ----------
    model : nn.Module
        Neural network model
    optimizer : torch.optim.Optimizer
        Search space optimizer (e.g. Adam)
    loss_fn :
        Loss function (e.g. nn.CrossEntropyLoss())
    train_dl :
        Iterable dataloader for training data.
    val_dl :
        Iterable dataloader for validation data.
    metrics: list
        List of sklearn metrics functions to be calculated
    metrics_name: list
        List of matrics names
    epochs : int
        Number of epochs to run
    device : string
        Specifies 'cuda' or 'cpu'
    task : string
        type of problem. It can be regression, binary or multiclass

    Returns
    -------
    Dictionary
        Similar to Keras' fit(), the output dictionary contains per-epoch
        history of training loss, training accuracy, validation loss, and
        validation accuracy.
    '''

    print('train() called: model=%s, opt=%s(lr=%f), epochs=%d, device=%s\n' % \
          (type(model).__name__, type(optimizer).__name__,
           optimizer.param_groups[0]['lr'], epochs, device))

    metrics = metrics if metrics else []
    metrics_name = metrics_name if metrics_name else [metric.__name__ for metric in metrics]

    history = {} # Collects per-epoch loss and metrics like Keras' fit().
    history['loss'] = []
    history['val_loss'] = []
    for name in metrics_name:
        history[name] = []
        history[f'val_{name}'] = []

    start_time_train = time.time()

    for epoch in range(epochs):

        # --- TRAIN AND EVALUATE ON TRAINING SET -----------------------------
        start_time_epoch = time.time()

        model.train()
        history_train = {name: 0 for name in ['loss']+metrics_name}

        for batch in train_dl:
            x    = batch[0].to(device)
            y    = batch[1].to(device)
            y_pred = model(x)
            loss = loss_fn(y_pred, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            y_pred = y_pred.detach().cpu().numpy()
            y = y.detach().cpu().numpy()


            history_train['loss'] += loss.item() * x.size(0)
            for name, func in zip(metrics_name, metrics):
              try:
                  history_train[name] += func(y, y_pred) * x.size(0)
              except:
                  if task == 'binary': y_pred_ = y_pred.round()
                  elif task == 'multiclass': y_pred_ = y_pred.argmax(axis=-1)
                  history_train[name] += func(y, y_pred_) * x.size(0)

        for name in history_train:
            history_train[name] /= len(train_dl.dataset)


        # --- EVALUATE ON VALIDATION SET -------------------------------------
        model.eval()
        history_val = {'val_' + name: 0 for name in metrics_name+['loss']}

        with torch.no_grad():
            for batch in val_dl:
                x    = batch[0].to(device)
                y    = batch[1].to(device)
                y_pred = model(x)
                loss = loss_fn(y_pred, y)

                y_pred = y_pred.cpu().numpy()
                y = y.cpu().numpy()

                history_val['val_loss'] += loss.item() * x.size(0)
                for name, func in zip(metrics_name, metrics):
                    try:
                        history_val['val_'+name] += func(y, y_pred) * x.size(0)
                    except:
                        if task == 'binary': y_pred_ = y_pred.round()
                        elif task == 'multiclass': y_pred_ = y_pred.argmax(axis=-1)

                        history_val['val_'+name] += func(y, y_pred_) * x.size(0)

        for name in history_val:
            history_val[name] /= len(val_dl.dataset)

        # PRINTING RESULTS

        end_time_epoch = time.time()

        for name in history_train:
            history[name].append(history_train[name])
            history['val_'+name].append(history_val['val_'+name])

        total_time_epoch = end_time_epoch - start_time_epoch

        print(f'Epoch {epoch+1:4d} {total_time_epoch:4.0f}sec', end='\t')
        for name in history_train:
            print(f'{name}: {history[name][-1]:10.3g}', end='\t')
            print(f"val_{name}: {history['val_'+name][-1]:10.3g}", end='\t')
        print()

    # END OF TRAINING LOOP

    end_time_train       = time.time()
    total_time_train     = end_time_train - start_time_train
    print()
    print('Time total:     %5.2f sec' % (total_time_train))

    return history

In [23]:
from sklearn.metrics import accuracy_score, roc_auc_score

history = train(model, optimizer, loss_fn, train_dl, train_dl,
                epochs=5,
                metrics=[accuracy_score],
                device=device,
                task='multiclass')

train() called: model=TextClassifier, opt=Adam(lr=0.001000), epochs=5, device=cuda

Epoch    1    4sec	loss:      0.372	val_loss:     0.0388	accuracy_score:       0.87	val_accuracy_score:      0.989	
Epoch    2    4sec	loss:     0.0327	val_loss:    0.00753	accuracy_score:      0.991	val_accuracy_score:          1	
Epoch    3    4sec	loss:    0.00591	val_loss:    0.00268	accuracy_score:          1	val_accuracy_score:          1	
Epoch    4    4sec	loss:     0.0026	val_loss:     0.0016	accuracy_score:          1	val_accuracy_score:          1	
Epoch    5    4sec	loss:    0.00146	val_loss:   0.000969	accuracy_score:          1	val_accuracy_score:          1	

Time total:     18.11 sec
